In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import time
import numpy as np
import pandas as pd
from multiprocessing import Pool
from optimize_alpha import optimize_alpha
from distance_calculations_source import distance
# from distance_calculations import distance

def compute_distance(dat, c, Rc, N, an, alpha):
    # pool = Pool()
    dist = distance(dat[:,c], dat, Rc[c], Rc, N, an, alpha, 0)
    return dist

def cluster_by_centroids(dat, d0, N, C, Rc, Rn):

    members = []
    members1 = []
    cluster_curr = 0
    cluster_assigned = np.zeros(C) - 1
    cluster_assigned1 = np.zeros(C) - 1

    dist_to_assigned = np.zeros(dat.shape[1], dtype=float) + d0

    dists = []

    s = time.time()
    a     = optimize_alpha(dat, Rc, Rn, N)
    an    = a * Rn / Rn.sum()
    alpha = an.sum()
    print("alpha optimized is:",a, "and took", time.time() - s, "seconds\n")
    s = time.time()
    iterate_ind = np.argsort(Rc)[::-1]

    for c in iterate_ind:
    # for c in np.arange(C):

        if cluster_assigned1[c] < 0 :
            ss = time.time()
            dist = compute_distance(dat, c, Rc, N, an, alpha)
            reassign_ind = np.intersect1d(np.nonzero(cluster_assigned < 0)[0], np.nonzero(np.array(dist < d0))[0])
            
            # print(len(reassign_ind), len(inds))
            members.append(reassign_ind)
            cluster_assigned[reassign_ind] = cluster_curr

            inds = np.nonzero(dist < dist_to_assigned)[0]
            dist_to_assigned[inds] = dist[inds]
            if len(inds) != 0: 
                members1.append(inds)
                cluster_assigned1[inds] = cluster_curr

            dists.append([dist])
            cluster_curr += 1


            # print(len(members[-1]), len(members1[-1]))
    # with open("/big/work/metadevol/benchmark_dataset1/initial_clusters_new", 'a') as file:
    #     for f in range(len(members)):
    #         for q in members[f]:
    #             file.write(str(q) + " " + str(f) + "\n")
    sum_t = 0
    sum_m1 = 0
    sum = 0
    for k in range(len(members1)):
        print("member size", len(members1[k]), len(np.nonzero(cluster_assigned1==k)[0]), len(members[k]))
        members1[k] = np.intersect1d(members1[k],np.nonzero(cluster_assigned1==k)[0])
        sum_t = sum_t + len(np.nonzero(cluster_assigned1==k)[0])
        sum_m1 = sum_m1 + len(members1[k])
        sum = sum + len(members[k])
    print(sum_t, sum_m1, sum)
    print("Obtained {} clusters from initial clustering".format(len(members)))
    print("Initial clustering took:", time.time() - s,"seconds")

    return members, np.array(dists).squeeze()


def count_numbers_of_sharedcontigs(dists, d1, min_shared_contigs):
    s = time.time()
    K, C = np.shape(dists)
    shared = np.zeros([K,K])
    # shared1 = np.zeros([K,K])
    neigh = np.array(dists < d1).astype(int)
    neigh_sum = neigh.sum(axis=0)
    for c in np.arange(C):
        if neigh_sum[c] >= 2:
            non_zeroind = neigh[:,c].nonzero()
            k_ind = np.array(np.meshgrid(non_zeroind,non_zeroind)).T.reshape(-1,2)
            # k_ind = k_ind[k_ind[:,0] != k_ind[:,1]]
            for i in k_ind:
                shared[i[0],i[1]] += 1
    shared = np.array(shared > min_shared_contigs).astype(int)

    # for c in np.arange(C):

    #     neigh = np.nonzero(np.array(dists)[:,c] < d1)[0]
    #     if len(neigh) >= 2:
    #         for k in neigh:
    #             for l in neigh:
    #                 shared[k,l] += 1

    # shared = np.array(shared > min_shared_contigs).astype(int)
    # print(shared1)

    links = []
    for k in np.arange(K):
        links.append(list(np.nonzero(shared[k])))
    print("count_numbers_of_sharedcontigs took ", time.time()-s, "seconds")
    return links


def recursive_set_allneighbors(k, component_curr, components, links):
    components[k] = component_curr
    for l in links[k][0]:

        if components[l] < 0 :

            recursive_set_allneighbors(l, component_curr, components, links)


def find_connected_components(links):
    s = time.time()
    K = np.shape(links)[0]
    components = np.zeros(K).astype(int) - 1
    component_curr = 0

    for k in np.arange(K):

        if components[k] < 0:

            recursive_set_allneighbors(k, component_curr, components, links)
            component_curr += 1

    if (component_curr != len(set(components))):
        raise Exception("problem with component calculations")
        exit()
    print("find_connected_components took ", time.time()-s, "seconds")
    num_components = component_curr
    numclust_incomponents = np.unique(components, return_counts=True)[1]
    print(numclust_incomponents)
    return components, num_components, numclust_incomponents


def merge_members_by_connnected_components(components, num_components, members):
    K = len(components)
    clusters = [[] for i in range(num_components)]

    for k in np.arange(K):
        clusters[components[k]].append(members[k])
 
    # for i in np.arange(num_components):
        # clusters[i] = np.unique(np.concatenate(clusters[i]).ravel())
       
    return clusters


def cluster_by_connecting_centroids(dat, d0, min_shared_contigs):
    s = time.time()
    contig_names = dat.columns
    dat = dat.to_numpy(dtype=np.float32)
    N, C = np.shape(dat)

    Rc    = dat.sum(axis=0)
    Rn    = dat.sum(axis=1)
    members, dists = cluster_by_centroids(dat, d0, N, C, Rc, Rn)
    # d1 = d0 * np.sqrt(N)
    d1 = d0 * 2
    # d1 = d0 
    links = count_numbers_of_sharedcontigs(dists, d1, min_shared_contigs)
    components, num_components, numclust_incomponents = find_connected_components(links)
    print("number of connected components", num_components)
    clusters = merge_members_by_connnected_components(components, num_components, members)

    print("count_by_connecting_centroids took ", time.time()-s, "seconds")
    return clusters, numclust_incomponents

if __name__ == "__main__":
    s = time.time()
    print("clustering initiated"+'\n')
    tmp_dir = "/big/work/metadevol/benchmark_dataset1/"
    # tmp_dir = "/big/work/metadevol/scripts/bamtools_api/build/"
    dat = pd.read_pickle(tmp_dir + 'X_pickle')
    d0 = 1
    min_shared_contigs = 5
    clusters, numclust_incomponents = cluster_by_connecting_centroids(dat, d0, min_shared_contigs)
    print("overall time taken for new clustering is: ", time.time()-s)


clustering initiated

alpha optimized is: 8.440742299720526 and took 1.7432684898376465 seconds

member size 1981 1970 1981
member size 1369 1369 1369
member size 1170 1157 1170
member size 1709 1709 1709
member size 2804 2679 2804
member size 3797 3797 3797
member size 1739 1716 1739
member size 8283 5647 8283
member size 1755 1755 1755
member size 2557 2557 2557
member size 2470 2470 2470
member size 1649 1167 145
member size 1164 1164 1164
member size 2433 2433 2433
member size 567 455 85
member size 1127 1127 1127
member size 1668 1668 1668
member size 1188 1188 1188
member size 3278 3105 3278
member size 1340 1246 219
member size 1051 1051 1040
member size 1197 1197 1197
member size 128 127 2
member size 1567 1567 1567
member size 175 175 2
member size 248 247 174
member size 116 116 4
member size 17 17 4
member size 26 26 3
member size 366 361 366
member size 33 33 1
member size 159 151 159
member size 294 283 294
member size 9 9 1
member size 14 14 3
member size 6 6 1
member siz

/home/yazhini/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [16]:
print(sum_t, sum_m1, sum)

NameError: name 'sum_t' is not defined

In [10]:
len(inds)

NameError: name 'inds' is not defined

In [ ]:
for f, fi in zip(clusters, range(len(clusters))):
    # f = map(str,f)
    print(np.shape(f))
    for q in f:
        with open("/big/work/metadevol/benchmark_dataset1/" + "connected_components_new", 'a') as file:
        # with open("/big/work/metadevol/scripts/bamtools_api/build/" + "connected_components_new", 'a') as file:
            for r in q:
                file.write(str(r) + " " + str(fi) + '\n')

In [ ]:
import numpy as np
np.min(dat)

In [ ]:
np.exp(15)

In [36]:
import time
import numpy as np
import pandas as pd
# from multiprocessing import Pool
from optimize_parameters import optimize_alpha
from distance_calculations_gammafunction import distance
# from distance_calculations import distance

def obtain_optimized_alpha(*argv):
    w_mat = np.array(argv[0])
    counts = np.array(argv[1])
    Rc = w_mat.sum(axis=0)
    Rn = w_mat.sum(axis=1)
    aw = optimize_alpha(w_mat, Rc, Rn, 4)
    aw = aw * counts / counts.sum()
    return aw

if __name__ == "__main__":
    s = time.time()
    print("clustering initiated"+'\n')
    tmp_dir = "/big/work/metadevol/benchmark_dataset1/"
    # tmp_dir = "/big/work/metadevol/scripts/bamtools_api/build/"
    dat = pd.read_pickle(tmp_dir + 'X_pickle')
    dat = np.array(dat)
    N, C = np.shape(dat) 
    d0 = 1
    min_shared_contigs = 5
    Rc = dat.sum(axis=0)
    Rn = dat.sum(axis=1)
    s = time.time()
    a     = optimize_alpha(dat, Rc, Rn, N)
    an    = a * Rn / Rn.sum()
    alpha = an.sum()
    print("alpha optimized is:",a, "and took", time.time() - s, "seconds\n")
    distr = []
    # for c in np.arange(C):
    #     distr.append(distance(dat[:,c], dat, Rc[c], Rc, N, an, alpha, 0))
    #     if c == 16617:
    #         print(distance(dat[:,c], dat, Rc[c], Rc, N, an, alpha, 0))

    kmer = pd.read_csv(tmp_dir + "kmer_counts", header=None)
    kmer = kmer.to_numpy()
    tetramer_counts = kmer.reshape(45760,256)
    tetramer_counts = (tetramer_counts / 2).transpose()

    trimercountsper_nt = np.split(tetramer_counts.sum(axis=1), 64)

    trimer_submatrices = np.vsplit(tetramer_counts, 64)
    Rc_kmers = np.array(trimer_submatrices).sum(axis=1)
    awa_values = np.array([])
    
    with Pool() as pool:
        awa_values = pool.starmap(obtain_optimized_alpha, [(x, y) for x, y in zip(trimer_submatrices, trimercountsper_nt)])
    
    awa_values = np.array(awa_values)
    alpha_values = awa_values.sum(axis=1)
    awa_values = awa_values.flatten()
    print(awa_values)
    # dist = []
    # for c in np.arange(np.shape(tetramer_counts)[1]):
    #     dist.append(distance(tetramer_counts[:,c], tetramer_counts[:,c], Rc_kmer[:,c], Rc_kmer[:,c], 256, awa_values, alpha_values, 1))
    print(time.time()-s)


clustering initiated

alpha optimized is: 8.445274503124452 and took 0.8412892818450928 seconds

[65.22040543 47.1864485  45.4985623  36.14973427 17.21545533 18.43637501
 15.80844796 16.37250452 29.39193912 20.15584564 29.15500607 28.53100593
 20.81132077 17.49222146 18.4170369  19.5301161  31.00646485 31.11921922
 20.18677541 21.07461887 11.55151021 18.80684603 14.16525629 13.46192623
 51.16119628 34.48034449 52.75031566 48.47356652 15.63879727 11.84962248
 15.98861683 14.0951357  35.04262459 29.36798715 21.2643935  21.12819982
 17.5658027  25.39579338 18.97260361 19.17378892 27.84308525 23.76384792
 22.5367284  30.593453   17.38873491 14.71548171 17.3165635  16.06592692
 23.57558881 20.74808658 21.65756739 16.72561733  7.88493088 15.72649325
 14.99322257 11.93926997 34.11144471 22.76810395 39.3396765  36.22597808
 18.41662639 15.26751019 21.57078773 13.79935398 93.31304796 72.22852302
 50.41925201 46.44344144 13.37765995 20.27405847 13.28663469 16.96846633
 75.124399   60.957275   63

In [37]:
Rc_kmers

array([[1.0000e+00, 2.3000e+01, 1.1650e+02, ..., 5.6000e+01, 0.0000e+00,
        2.7500e+01],
       [0.0000e+00, 1.5000e+01, 1.1000e+02, ..., 4.5500e+01, 1.0000e+00,
        2.2000e+01],
       [8.5000e+00, 1.5100e+02, 5.4000e+01, ..., 3.1450e+02, 4.0000e+00,
        5.3000e+01],
       ...,
       [4.9000e+01, 5.4950e+02, 2.8000e+01, ..., 5.4800e+02, 3.6500e+01,
        7.3000e+01],
       [1.1000e+02, 1.6135e+03, 1.0000e+00, ..., 6.0650e+02, 1.1900e+02,
        1.4350e+02],
       [7.2500e+01, 1.1155e+03, 5.5000e+00, ..., 5.6150e+02, 6.9000e+01,
        5.7500e+01]])

In [ ]:
np.max(np.array(distr))

In [ ]:
len(dist)

In [ ]:
3.12775078e+01,  3.47597542e+01,  2.73754501e+01,  2.89249573e+01

In [ ]:
kmer = pd.read_csv(tmp_dir + "kmer_counts", header=None)
kmer = kmer.to_numpy()
tetramer_counts = kmer.reshape(45760,256)
tetramer_counts = (tetramer_counts / 2)




In [ ]:
np.shape(Rc_kmer)

In [ ]:
awa_values